In [1]:
%cd ..

E:\py_strict_list


In [52]:
from functools import reduce

In [2]:
from develops import StructureStrictList, StructureInvalidError

In [53]:
a = [1,2,3]
reduce(lambda x,y:x+y, a)

6

In [54]:
a = ["a","b","c"]
reduce(lambda x,y:x+y, a)

'abc'

In [78]:
def flatten(_list):
    def flatten_gen(_list):
        for item in _list:
            if isinstance(item, list):
                yield from flatten_gen(item)
            else:
                yield item
    return list(flatten_gen(_list))

In [55]:
class StructureListValidator():
    """
    StructureListのValidator(ディスクリプタ)
    セットされるときに構造チェックを行う
    """
    def __init__(self, include_outer_length=False):
        """
        include_outer_length: bool
            最も外側を含めるかどうか
        """
        self.include_outer_length = include_outer_length
        
    def __set_name__(self, owner, name):
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        """
        privateを参照
        """
        if obj is None:  # クラスアトリビュートとして参照された時
            return self
        
        return getattr(obj, self.private_name)

    def __set__(self, obj, list_like):
        """
        構造チェック・hook関数の実行と移行
        """
        if not getattr(obj, self.private_name).check_same_structure_with(list_like,
                                                                         include_outer_length=self.include_outer_length
                                                                        ):
            raise StructureInvalidError("This list_like is invalid")

        ssl_list_like = type(getattr(obj, self.private_name)).from_list(list_like)
        pre_hook_func = getattr(obj, self.private_name).hook_func
        setattr(obj, self.private_name, ssl_list_like)
        pre_hook_func()  # Hook関数を実行
        getattr(obj, self.private_name).hook_func = pre_hook_func

In [80]:
class A:
    inner_list = StructureListValidator(include_outer_length=False)
    def __init__(self, list_like):
        self._inner_list = StructureStrictList.from_list(list_like)
        self.inner_list = self._inner_list
        self.make_sum()
        self._inner_list.hook_func.add(self.make_sum)
        
    def make_sum(self):
        self.sum = reduce(lambda x,y:x+y, flatten(self._inner_list))

In [81]:
a = A([1,2,3,4])

In [82]:
b = A(["a","b","c"])

In [83]:
a.inner_list

[1, 2, 3, 4]

In [72]:
a.inner_list.append(10)
a.sum

20

In [73]:
a.inner_list = [[1,2]]

StructureInvalidError: This list_like is invalid

In [74]:
b.inner_list.append("d")
b.sum

'abcd'

In [21]:
def strict_list_property(private_name, include_outer_length=False):
    """
    StructureStrictListをpropertyに登録する関数
    
    private_name: str
        propertyで隠す変数
    include_outer_lengh: bool
        もっとも外側の長さを比較に含めるかどうか
    
    """
    def getter(instance):
        return instance.__dict__[private_name]

    def setter(instance, list_like):
        if not instance.__dict__[private_name].check_same_structure_with(list_like,
                                                                         include_outer_length=include_outer_length
                                                                         ):
            raise StructureInvalidError("This list-like is invalid")
        previous_hook_func = instance.__dict__[private_name].hook_func
        instance.__dict__[private_name] = type(instance.__dict__[private_name]).from_list(list_like)
        previous_hook_func()  # hook_funcの実行
        instance.__dict__[private_name].hook_func = previous_hook_func
        
    return property(getter, setter)

In [22]:
class B:
    def __init__(self, list_like):
        self._inner_list = StructureStrictList.from_list(list_like)
        self.make_sum()
        self._inner_list.hook_func.add(self.make_sum)
        
    def make_sum(self):
         self.sum = reduce(lambda x,y:x+y, self._inner_list)
        
    
    inner_list = strict_list_property("_inner_list", False)

In [23]:
b = B([1,2,3,3])

In [24]:
b.inner_list

[1, 2, 3, 3]

In [25]:
b.sum

9

In [26]:
b.inner_list.append(10)

In [27]:
b.sum

19

In [28]:
b.inner_list = [1,2]

In [29]:
b.sum

3

In [30]:
b.inner_list = ["a"]

Exception: This list-like is invalid